In [1]:
import pandas as pd

#### leverage logistic regression as this is classification

In [2]:
#load in data
df = pd.read_csv('lagging.csv')

C:\ProgramData\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:2785: DtypeWarning: Columns (9,10,11,12,13,14,27,28,29) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [3]:
#import classification, traintestsplit, and counter
from collections import Counter
from sklearn.datasets import make_classification
import numpy as np
from sklearn.model_selection import train_test_split

## Create Dynamic Classifications Through Scoring System as to what is considred a large jump

In [4]:
firesizecriteria=[]
for i,y in enumerate(df['FIRE SIZE MAX']):

    if df['diffmax'][i] > 20000:
            firesizecriteria.append(1)
    elif df['diffmax'][i]*5 < df['FIRE SIZE MAX'][i]:
        if df['diffmax'][i] > 600:
            firesizecriteria.append(1)
        else:
            firesizecriteria.append(0)
    elif df['diffmax'][i]*2 < float(df['FIRE SIZE MAX'][i]):
        if df['diffmax'][i] > 1000:
            firesizecriteria.append(1)
        else:
            firesizecriteria.append(0)

            
    elif df['diffmax'][i]*4 < float(df['FIRE SIZE MAX'][i]):
        if df['diffmax'][i] > 500:
            firesizecriteria.append(1)
        else:
            firesizecriteria.append(0)
    elif df['diffmax'][i]*100 < float(df['FIRE SIZE MAX'][i]):
        if df['diffmax'][i] > 200:
            firesizecriteria.append(1)
        else:
            firesizecriteria.append(0)
    
    else:
        firesizecriteria.append(0)

#     else:
#         print(0)   
#         firesizecriteria.append(0)

### Drop Unncessary Columns

In [5]:
df=pd.concat([df, pd.DataFrame(firesizecriteria).rename(columns={0:'firesize_criteria'})], axis=1)

In [6]:
df5 = pd.get_dummies(df['day of week'], dummy_na=0)

In [7]:
df=pd.concat([df,df5], axis=1)

In [8]:
df55=df.dropna()

In [9]:
df55.columns

Index(['Unnamed: 0', 'Unnamed: 0.1', 'Unnamed: 0.1.1', 'Unnamed: 0.1.1.1',
       'index', 'Unnamed: 0.1.1.1.1', 'dates', 'latit', 'longit', 'longitude',
       'latitude', 'state', 'sz_class', 'cause', 'firesize', 'A', 'B', 'C',
       'D', 'E', 'F', 'G', 'FIRE SIZE SUM', 'FIRE SIZE MAX', 'diffmax',
       'fire binary', 'weekend', 'day', 'day of week', 'lat lon', '0', '0.1',
       '0.2', '0.3', '0.4', '0.5', 'df_30_60_sum', 'df_30_60_max',
       'df_30_B_60_sum', 'df_30_A_60_sum', 'df_30_C_60_sum', 'df_30_D_60_sum',
       'df_30_E_60_sum', 'df_30_F_60_sum', 'df_30_G_60_sum', 'y4sum', 'y4max',
       'y1sum', 'y1max', 'd45sum', 'd45max', 'df_30_60_sum.1',
       'df_30_60_max.1', 'df_30_B_60_sum.1', 'df_30_A_60_sum.1',
       'df_30_C_60_sum.1', 'df_30_D_60_sum.1', 'df_30_E_60_sum.1',
       'df_30_F_60_sum.1', 'df_30_G_60_sum.1', 'firesize_criteria', 'Friday',
       'Monday', 'Saturday', 'Sunday', 'Thursday', 'Tuesday', 'Wednesday'],
      dtype='object')

In [10]:
df55=df55.sort_values('dates')

In [11]:
df55=df55.reset_index().drop(columns='level_0')

# Estabolish a y variable

In [12]:
y = df55['firesize_criteria']

In [13]:
df55=df55.drop(columns=['lat lon', 'Unnamed: 0', 'Unnamed: 0.1', 'Unnamed: 0.1.1', 'Unnamed: 0.1.1.1', 'index', 'Unnamed: 0.1.1.1.1'])

In [14]:
df55=df55.drop(columns=['day', 'firesize_criteria'])

In [15]:
df55=df55.drop(columns=['day of week', 'cause', 'sz_class', 'state', 'longitude', 'latitude', 'firesize', 'df_30_A_60_sum.1', 'df_30_B_60_sum.1', 'df_30_C_60_sum.1', 'df_30_D_60_sum.1', 'df_30_E_60_sum.1', 'df_30_F_60_sum.1', 'df_30_G_60_sum.1', 'df_30_60_sum.1', 'df_30_60_max.1'])

# Create X and Y variables split on Date. 

### This is to train based on dates. Test is assumed to be in the future.

In [16]:
X_train=df55[:int(df55.shape[0]*.7)]
X_test=df55[int(df55.shape[0]*.7):]

##### Standardize The Data

In [17]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
scaler.fit(X_train)
X_train=scaler.transform(X_train)
X_test=scaler.transform(X_test)

In [18]:
y_train=y[:int(df55.shape[0]*.7)]
y_test=y[int(df55.shape[0]*.7):]

In [19]:
y_train.shape

(98119,)

In [20]:
df55.shape

(140171, 43)

# Make classifications of Data

In [21]:
X_train, y_train = make_classification(n_classes=2, class_sep=2, weights=[.7,.3], n_samples=1700 )

In [22]:
X_test, y_test = make_classification(n_classes=2, class_sep=2, weights=[.7,.3], n_samples=300 )

# Run Keras to Use Neural Network that Classifies

In [23]:
from imblearn.under_sampling import RandomUnderSampler # doctest: +NORMALIZE_WHITESPACE
rus = RandomUnderSampler(ratio = .5)
rus.fit(X_train, y_train)

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\ensemble\weight_boosting.py:29: DeprecationWarning: numpy.core.umath_tests is an internal NumPy module and should not be imported. It will be removed in a future NumPy release.
  from numpy.core.umath_tests import inner1d
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\utils\deprecation.py:77: DeprecationWarning: Function _ratio_float is deprecated; Use a float for 'ratio' is deprecated from version 0.2. The support will be removed in 0.4. Use a dict, str, or a callable instead.
  warnings.warn(msg, category=DeprecationWarning)


RandomUnderSampler(random_state=None, ratio=0.5, replacement=False,
          return_indices=False)

In [24]:
from imblearn.under_sampling import RandomUnderSampler # doctest: +NORMALIZE_WHITESPACE
rus = RandomUnderSampler(ratio = .5)
rus.fit(X_test, y_test)

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\utils\deprecation.py:77: DeprecationWarning: Function _ratio_float is deprecated; Use a float for 'ratio' is deprecated from version 0.2. The support will be removed in 0.4. Use a dict, str, or a callable instead.
  warnings.warn(msg, category=DeprecationWarning)


RandomUnderSampler(random_state=None, ratio=0.5, replacement=False,
          return_indices=False)

In [25]:
from keras.utils.np_utils import to_categorical

C:\ProgramData\Anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


Couldn't import dot_parser, loading of dot files will not be possible.


In [26]:
from keras.models import Sequential
from keras.layers import Dense

In [27]:
model = Sequential()

In [28]:
from keras.utils.np_utils import to_categorical
y_train = to_categorical(y_train)
y_test = to_categorical(y_test)

In [29]:
###put in input, outputs and hidden sizes
n_input = X_train.shape[1]
n_hidden = n_input
n_output = y_train.shape[1]

In [30]:
from keras.layers import Dropout

In [31]:
#add key features
model.add(Dropout(0.2))
model.add(Dense(n_hidden, input_dim=500, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(n_hidden, input_dim=100, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(n_hidden, input_dim=1000, activation='relu'))
model.add(Dense(n_output, activation='sigmoid'))

In [32]:
#design model
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

In [33]:
#enter in keras model
history = model.fit(X_train, y_train, validation_data=(X_test, y_test),
                    epochs=25, batch_size=None)#, batch_size=None, verbose=0)


Train on 1700 samples, validate on 300 samples
Epoch 1/25
1700/1700 [==============================] - 1s 334us/step - loss: 0.7174 - acc: 0.5397 - val_loss: 0.6119 - val_acc: 0.6783
Epoch 2/25
1700/1700 [==============================] - 0s 57us/step - loss: 0.5851 - acc: 0.6774 - val_loss: 0.5321 - val_acc: 0.7067
Epoch 3/25
1700/1700 [==============================] - 0s 54us/step - loss: 0.5158 - acc: 0.7159 - val_loss: 0.4876 - val_acc: 0.7233
Epoch 4/25
1700/1700 [==============================] - 0s 58us/step - loss: 0.4316 - acc: 0.7662 - val_loss: 0.4396 - val_acc: 0.7750
Epoch 5/25
1700/1700 [==============================] - 0s 51us/step - loss: 0.3668 - acc: 0.8147 - val_loss: 0.3954 - val_acc: 0.8300
Epoch 6/25
1700/1700 [==============================] - 0s 56us/step - loss: 0.3160 - acc: 0.8582 - val_loss: 0.3671 - val_acc: 0.8633
Epoch 7/25
1700/1700 [==============================] - 0s 60us/step - loss: 0.2731 - acc: 0.8879 - val_loss: 0.3546 - val_acc: 0.8467
Epoch 8

In [34]:
from sklearn.metrics import classification_report, confusion_matrix

In [35]:
confusion_matrix(np.argmax(y_test, axis=1), model.predict_classes(X_test))

array([[189,  19],
       [ 27,  65]], dtype=int64)